In [1]:
import pandas as pd

In [82]:
from sklearn.preprocessing import StandardScaler

In [2]:
#data read
train = pd.read_csv(r'sales_train.csv')
test = pd.read_csv(r'test_data.csv')
outlet_master = pd.read_csv(r'outlet_master.csv')
item_master = pd.read_csv(r'item_master.csv')

In [20]:
train.columns

Index(['outletCode', 'itemCode', 'sales_in_litres', 'invoice_date_year',
       'invoice_date_month', 'invoice_date_day'],
      dtype='object')

In [27]:
train.head()

,outletCode,itemCode,sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day
0,customer_id_1,sku_id_51,44.0,2019,11,20
1,customer_id_2,sku_id_51,22.0,2019,4,13
2,customer_id_5,sku_id_51,22.0,2019,6,4
3,customer_id_5,sku_id_51,22.0,2019,7,5
4,customer_id_5,sku_id_51,22.0,2019,1,6


In [90]:
test.head()

,outletCode,invoice_date,itemCode,actual_sales_in_litres,predicted_sales_in_litres
0,customer_id_10,12/10/2019,sku_id_48,5,NaN
1,customer_id_10,14/12/2019,sku_id_53,5,NaN
2,customer_id_10,08/01/2020,sku_id_48,5,NaN
3,customer_id_1000,21/10/2020,sku_id_14,4,NaN
4,customer_id_1000,21/10/2020,sku_id_11,12,NaN


In [111]:
outlet_master.head()


,outletCode,latitude,longitude,outletType,outletCategory
0,customer_id_1535,30.162201,31.282801,type_6,category_1
1,customer_id_1640,30.082500,31.192900,type_8,category_1
2,customer_id_1631,30.117599,31.253700,type_5,category_1
3,customer_id_625,30.135500,31.384899,type_4,category_1
4,customer_id_1605,30.146700,31.145100,type_5,category_1


In [39]:
outlet_master['outletType'].unique()

array(['type_6', 'type_8', 'type_5', 'type_4', 'type_3', 'type_1',
       'type_7', 'type_2'], dtype=object)

In [7]:
item_master.head()

,itemCode,packet_size,brand,category,class
0,sku_id_1,1.0,brand_1,sku_category_1,sku_class_1
1,sku_id_38,209.0,brand_9,sku_category_15,sku_class_1
2,sku_id_40,209.0,brand_9,sku_category_16,sku_class_1
3,sku_id_42,22.0,brand_9,sku_category_16,sku_class_1
4,sku_id_43,5.0,brand_9,sku_category_16,sku_class_1


In [37]:
item_master['class'].unique()

array(['sku_class_1', 'sku_class_2'], dtype=object)

In [9]:
train.dtypes

outletCode          object
invoice_date        object
itemCode            object
sales_in_litres    float64
dtype: object

In [10]:
train.isna().sum()
train.isnull().sum()

outletCode         0
invoice_date       0
itemCode           0
sales_in_litres    0
dtype: int64

In [11]:
train.shape

(283545, 4)

In [12]:
{column: len(train[column].unique()) for column in train.columns}

{'outletCode': 2780,
 'invoice_date': 630,
 'itemCode': 71,
 'sales_in_litres': 444}

In [91]:
def encode_date(df,column):
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df= df.drop(column,axis=1)
    return df

In [14]:
#extract date featured
train = encode_date(train,column = 'invoice_date')


In [94]:
test = encode_date(test,column = 'invoice_date')


In [95]:
test.head()

,outletCode,itemCode,actual_sales_in_litres,predicted_sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day
0,customer_id_10,sku_id_48,5,NaN,2019,12,10
1,customer_id_10,sku_id_53,5,NaN,2019,12,14
2,customer_id_10,sku_id_48,5,NaN,2020,8,1
3,customer_id_1000,sku_id_14,4,NaN,2020,10,21
4,customer_id_1000,sku_id_11,12,NaN,2020,10,21


In [102]:
test = test.drop('predicted_sales_in_litres',axis=1)


In [103]:
test.head()

,outletCode,itemCode,actual_sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day
0,customer_id_10,sku_id_48,5,2019,12,10
1,customer_id_10,sku_id_53,5,2019,12,14
2,customer_id_10,sku_id_48,5,2020,8,1
3,customer_id_1000,sku_id_14,4,2020,10,21
4,customer_id_1000,sku_id_11,12,2020,10,21


In [104]:
train.head()

,sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,...,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_67,itemCode_sku_id_68,itemCode_sku_id_69,itemCode_sku_id_7,itemCode_sku_id_70,itemCode_sku_id_71,itemCode_sku_id_8,itemCode_sku_id_9
0,44.0,2019,11,20,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22.0,2019,4,13,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22.0,2019,6,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22.0,2019,7,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.0,2019,1,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
#mapping data from outlet_master to train
out = pd.DataFrame()
u = train.sort_values('outletCode')
v = outlet_master.sort_values('outletCode')[['outletCategory']].reset_index(drop=1)
out = pd.concat([u, v], 1)


In [42]:
#mapping data from outlet_master to train
out0 = pd.DataFrame()
u = train.sort_values('outletCode')
v = outlet_master.sort_values('outletCode')[['outletType']].reset_index(drop=1)
out0 = pd.concat([u, v], 1)


In [43]:
#mapping data from item_master to train
out1 = pd.DataFrame()
u = train.sort_values('itemCode')
v = item_master.sort_values('itemCode')[['class']].reset_index(drop=1)
out1 = pd.concat([u, v], 1)

In [31]:
train['outletCategory'] = out['outletCategory']
train.head()

,outletCode,itemCode,sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory
0,customer_id_1,sku_id_51,44.0,2019,11,20,category_2
1,customer_id_2,sku_id_51,22.0,2019,4,13,category_2
2,customer_id_5,sku_id_51,22.0,2019,6,4,category_2
3,customer_id_5,sku_id_51,22.0,2019,7,5,category_2
4,customer_id_5,sku_id_51,22.0,2019,1,6,category_2


In [44]:
train['outletType'] = out0['outletType']
train['class'] = out1['class']
train.head()

,outletCode,itemCode,sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory,outletType,class
0,customer_id_1,sku_id_51,44.0,2019,11,20,category_2,type_6,sku_class_1
1,customer_id_2,sku_id_51,22.0,2019,4,13,category_2,type_5,sku_class_1
2,customer_id_5,sku_id_51,22.0,2019,6,4,category_2,type_5,sku_class_1
3,customer_id_5,sku_id_51,22.0,2019,7,5,category_2,type_5,sku_class_1
4,customer_id_5,sku_id_51,22.0,2019,1,6,category_2,type_5,sku_class_1


In [120]:
out2 = pd.DataFrame()
v = outlet_master.sort_values('outletCode')[['outletCategory']].reset_index(drop=1)
u = test.sort_values('outletCode')
out2 = pd.concat([u, v], 1)

out3 = pd.DataFrame()
u = test.sort_values('outletCode')
v = outlet_master.sort_values('outletCode')[['outletType']].reset_index(drop=1)
out3 = pd.concat([u, v], 1)

out4 = pd.DataFrame()
u = test.sort_values('itemCode')
v = item_master.sort_values('itemCode')[['class']].reset_index(drop=1)
out4 = pd.concat([u, v], 1)


test['outletCategory'] = out2['outletCategory']
test['outletType'] = out3['outletType']
test['class'] = out4['class']

In [121]:
test.head()

,outletCode,itemCode,actual_sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory,outletType,class
0,customer_id_10,sku_id_48,5,2019,12,10,category_2,type_6,sku_class_1
1,customer_id_10,sku_id_53,5,2019,12,14,category_2,type_5,sku_class_1
2,customer_id_10,sku_id_48,5,2020,8,1,category_2,type_5,sku_class_1
3,customer_id_1000,sku_id_14,4,2020,10,21,category_2,type_5,sku_class_1
4,customer_id_1000,sku_id_11,12,2020,10,21,category_2,type_5,sku_class_1


In [55]:
train.dtypes

sales_in_litres       float64
invoice_date_year       int64
invoice_date_month      int64
invoice_date_day        int64
outletCategory         object
outletType             object
class                  object
dtype: object

In [54]:
train= train.drop(['outletCode','itemCode'],axis=1)
train.head()

,sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory,outletType,class
0,44.0,2019,11,20,category_2,type_6,sku_class_1
1,22.0,2019,4,13,category_2,type_5,sku_class_1
2,22.0,2019,6,4,category_2,type_5,sku_class_1
3,22.0,2019,7,5,category_2,type_5,sku_class_1
4,22.0,2019,1,6,category_2,type_5,sku_class_1


In [122]:
test= test.drop(['outletCode','itemCode'],axis=1)
test.head()

,actual_sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory,outletType,class
0,5,2019,12,10,category_2,type_6,sku_class_1
1,5,2019,12,14,category_2,type_5,sku_class_1
2,5,2020,8,1,category_2,type_5,sku_class_1
3,4,2020,10,21,category_2,type_5,sku_class_1
4,12,2020,10,21,category_2,type_5,sku_class_1


In [58]:
train['itemCode'] = out1['itemCode']
train.head()

,sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory,outletType,class,itemCode
0,44.0,2019,11,20,category_2,type_6,sku_class_1,sku_id_51
1,22.0,2019,4,13,category_2,type_5,sku_class_1,sku_id_51
2,22.0,2019,6,4,category_2,type_5,sku_class_1,sku_id_51
3,22.0,2019,7,5,category_2,type_5,sku_class_1,sku_id_51
4,22.0,2019,1,6,category_2,type_5,sku_class_1,sku_id_51


In [123]:
test['itemCode'] = out2['itemCode']
test.head()

,actual_sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory,outletType,class,itemCode
0,5,2019,12,10,category_2,type_6,sku_class_1,sku_id_48
1,5,2019,12,14,category_2,type_5,sku_class_1,sku_id_53
2,5,2020,8,1,category_2,type_5,sku_class_1,sku_id_48
3,4,2020,10,21,category_2,type_5,sku_class_1,sku_id_14
4,12,2020,10,21,category_2,type_5,sku_class_1,sku_id_11


In [62]:
for column in train:
    a= train[column].unique()
    print(a)
    

[4.400e+01 2.200e+01 6.600e+01 1.100e+02 4.400e+03 2.200e+03 1.760e+02
 1.100e+03 8.800e+01 5.500e+02 8.800e+02 9.680e+02 2.200e+02 3.080e+02
 2.860e+02 1.320e+02 4.400e+02 1.540e+02 3.300e+02 2.640e+02 6.600e+02
 3.190e+03 1.166e+03 2.420e+02 4.158e+03 1.980e+02 3.960e+02 4.840e+02
 3.520e+02 6.380e+02 7.040e+02 7.700e+02 1.342e+03 1.012e+03 7.480e+02
 3.740e+02 1.540e+03 4.180e+02 1.430e+03 9.900e+02 1.980e+03 3.300e+03
 1.936e+03 1.320e+03 1.650e+03 5.720e+02 6.820e+02 5.280e+02 1.056e+03
 1.144e+03 2.090e+03 6.886e+03 1.034e+03 1.474e+03 2.640e+03 2.420e+03
 5.060e+02 6.600e+03 1.210e+03 3.850e+03 2.750e+03 7.260e+02 1.122e+03
 4.620e+02 5.940e+02 1.078e+03 2.178e+03 3.718e+03 1.500e+01 3.000e+01
 4.500e+01 2.500e+01 5.000e+00 1.000e+01 1.050e+02 6.000e+01 7.500e+01
 3.000e+02 4.500e+02 2.250e+02 2.100e+02 3.750e+02 6.000e+02 1.500e+02
 1.200e+02 1.350e+02 7.500e+02 1.950e+02 7.950e+02 5.250e+02 9.000e+02
 1.500e+03 9.000e+01 5.000e+01 4.000e+01 9.500e+01 2.000e+01 3.500e+01
 1.800

In [128]:
#onehotencode
def onehot_encoder(df,column):
    dummies = pd.get_dummies(train[column], prefix=column)
    df = pd.concat([df,dummies],axis=1)
    df= df.drop(column,axis=1)
    return df

In [59]:
for column in ['outletCategory','outletType','class','itemCode']:
    train = onehot_encoder(train,column= column) 

In [129]:
def onehot_encoder2(df,column):
    dummies2 = pd.get_dummies(test[column], prefix=column)
    df = pd.concat([df,dummies2],axis=1)
    df= df.drop(column,axis=1)
    return df

In [130]:
for column in ['outletCategory','outletType','class','itemCode']:
    test = onehot_encoder2(test,column= column) 

In [64]:
train.dtypes

sales_in_litres              float64
invoice_date_year              int64
invoice_date_month             int64
invoice_date_day               int64
outletCategory_category_1      uint8
                              ...   
itemCode_sku_id_7              uint8
itemCode_sku_id_70             uint8
itemCode_sku_id_71             uint8
itemCode_sku_id_8              uint8
itemCode_sku_id_9              uint8
Length: 87, dtype: object

In [132]:
test.head()

,actual_sales_in_litres,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,...,itemCode_sku_id_54,itemCode_sku_id_55,itemCode_sku_id_56,itemCode_sku_id_57,itemCode_sku_id_58,itemCode_sku_id_60,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_68,itemCode_sku_id_8
0,5,2019,12,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,2019,12,14,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,2020,8,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2020,10,21,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,2020,10,21,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
X_train = train.drop(['sales_in_litres'],axis=1)
X_train.head()

<ipython-input-67-9cf9b2ec51a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train['sales_in_litres'] = train['sales_in_litres']


,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,outletType_type_5,...,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_67,itemCode_sku_id_68,itemCode_sku_id_69,itemCode_sku_id_7,itemCode_sku_id_70,itemCode_sku_id_71,itemCode_sku_id_8,itemCode_sku_id_9
0,2019,11,20,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019,4,13,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2019,6,4,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2019,7,5,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2019,1,6,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [78]:
Y_train= train['sales_in_litres']
Y_train.head()

0    44.0
1    22.0
2    22.0
3    22.0
4    22.0
Name: sales_in_litres, dtype: float64

In [133]:
X_test = test.drop('actual_sales_in_litres',axis=1)


In [147]:
Y_actual = test['actual_sales_in_litres']
Y_actual.head()

0     5
1     5
2     5
3     4
4    12
Name: actual_sales_in_litres, dtype: int64

In [134]:
X_test.head()

,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,outletType_type_5,...,itemCode_sku_id_54,itemCode_sku_id_55,itemCode_sku_id_56,itemCode_sku_id_57,itemCode_sku_id_58,itemCode_sku_id_60,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_68,itemCode_sku_id_8
0,2019,12,10,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019,12,14,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2020,8,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2020,10,21,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2020,10,21,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [79]:
X_train.describe()

,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,outletType_type_5,...,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_67,itemCode_sku_id_68,itemCode_sku_id_69,itemCode_sku_id_7,itemCode_sku_id_70,itemCode_sku_id_71,itemCode_sku_id_8,itemCode_sku_id_9
count,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,...,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000,283545.000000
mean,2019.461870,6.256139,15.179308,0.000462,0.009342,0.000335,0.000025,0.000455,0.001062,0.005375,...,0.012400,0.017905,0.002243,0.000014,0.000293,0.000007,0.000035,0.000018,0.001735,0.000004
std,0.498545,3.283565,8.818253,0.021489,0.096204,0.018301,0.004969,0.021325,0.032564,0.073116,...,0.110664,0.132608,0.047308,0.003756,0.017107,0.002656,0.005939,0.004199,0.041619,0.001878
min,2019.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019.000000,3.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019.000000,6.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2020.000000,9.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,12.000000,31.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [144]:
X_test.describe()

,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,outletType_type_5,...,itemCode_sku_id_54,itemCode_sku_id_55,itemCode_sku_id_56,itemCode_sku_id_57,itemCode_sku_id_58,itemCode_sku_id_60,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_68,itemCode_sku_id_8
count,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,...,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000
mean,2019.982748,8.799691,18.179243,0.011895,0.240534,0.008626,0.000636,0.011713,0.027331,0.138382,...,0.002633,0.000091,0.000272,0.011169,0.002633,0.013802,0.023336,0.006992,0.000272,0.000454
std,0.130216,2.397596,6.922710,0.108419,0.427427,0.092480,0.025204,0.107598,0.163054,0.345316,...,0.051250,0.009529,0.016503,0.105095,0.051250,0.116673,0.150975,0.083328,0.016503,0.021304
min,2019.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020.000000,8.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020.000000,10.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2020.000000,10.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,12.000000,31.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [86]:
#data scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train),columns=X_train.columns)
X_train.describe()

,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,outletType_type_5,...,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_67,itemCode_sku_id_68,itemCode_sku_id_69,itemCode_sku_id_7,itemCode_sku_id_70,itemCode_sku_id_71,itemCode_sku_id_8,itemCode_sku_id_9
count,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,...,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05,2.835450e+05
mean,1.461687e-13,-7.547047e-15,-2.441930e-15,-6.230120e-14,7.506472e-14,-3.865472e-14,-2.054761e-15,-6.157153e-14,-5.085700e-14,-4.837255e-13,...,-1.481386e-13,-3.200750e-13,-1.141528e-13,-2.734287e-14,-9.764160e-15,-3.019256e-15,-1.909053e-14,1.956825e-14,-1.158720e-13,-4.194765e-15
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-9.264383e-01,-1.600744e+00,-1.607953e+00,-2.149933e-02,-9.711095e-02,-1.830727e-02,-4.968707e-03,-2.133451e-02,-3.259890e-02,-7.351090e-02,...,-1.120529e-01,-1.350255e-01,-4.741385e-02,-3.755970e-03,-1.711164e-02,-2.655862e-03,-5.938772e-03,-4.199309e-03,-4.169161e-02,-1.877975e-03
25%,-9.264383e-01,-9.916493e-01,-9.275445e-01,-2.149933e-02,-9.711095e-02,-1.830727e-02,-4.968707e-03,-2.133451e-02,-3.259890e-02,-7.351090e-02,...,-1.120529e-01,-1.350255e-01,-4.741385e-02,-3.755970e-03,-1.711164e-02,-2.655862e-03,-5.938772e-03,-4.199309e-03,-4.169161e-02,-1.877975e-03
50%,-9.264383e-01,-7.800658e-02,-2.033381e-02,-2.149933e-02,-9.711095e-02,-1.830727e-02,-4.968707e-03,-2.133451e-02,-3.259890e-02,-7.351090e-02,...,-1.120529e-01,-1.350255e-01,-4.741385e-02,-3.755970e-03,-1.711164e-02,-2.655862e-03,-5.938772e-03,-4.199309e-03,-4.169161e-02,-1.877975e-03
75%,1.079403e+00,8.356362e-01,8.868769e-01,-2.149933e-02,-9.711095e-02,-1.830727e-02,-4.968707e-03,-2.133451e-02,-3.259890e-02,-7.351090e-02,...,-1.120529e-01,-1.350255e-01,-4.741385e-02,-3.755970e-03,-1.711164e-02,-2.655862e-03,-5.938772e-03,-4.199309e-03,-4.169161e-02,-1.877975e-03
max,1.079403e+00,1.749279e+00,1.794088e+00,4.651307e+01,1.029750e+01,5.462311e+01,2.012596e+02,4.687241e+01,3.067589e+01,1.360342e+01,...,8.924360e+00,7.406006e+00,2.109089e+01,2.662428e+02,5.843975e+01,3.765256e+02,1.683850e+02,2.381344e+02,2.398564e+01,5.324885e+02


In [145]:
scaler.fit(X_test)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)
X_test.describe()

,invoice_date_year,invoice_date_month,invoice_date_day,outletCategory_category_1,outletCategory_category_2,outletType_type_1,outletType_type_2,outletType_type_3,outletType_type_4,outletType_type_5,...,itemCode_sku_id_54,itemCode_sku_id_55,itemCode_sku_id_56,itemCode_sku_id_57,itemCode_sku_id_58,itemCode_sku_id_60,itemCode_sku_id_65,itemCode_sku_id_66,itemCode_sku_id_68,itemCode_sku_id_8
count,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,...,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04,1.101300e+04
mean,6.212511e-13,1.548727e-15,-5.750266e-16,2.207934e-14,2.652341e-14,1.655009e-14,-2.288150e-15,1.632100e-14,-2.267015e-14,-3.218952e-14,...,-1.978091e-15,6.771209e-16,7.270711e-16,3.850875e-16,-1.307124e-15,-7.259899e-16,2.709517e-15,-5.616047e-15,-1.903826e-16,1.669229e-15
std,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,...,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00
min,-7.547394e+00,-3.253277e+00,-2.481690e+00,-1.097189e-01,-5.627739e-01,-9.328037e-02,-2.521937e-02,-1.088681e-01,-1.676285e-01,-4.007581e-01,...,-5.138291e-02,-9.529429e-03,-1.650696e-02,-1.062768e-01,-5.138291e-02,-1.183006e-01,-1.545757e-01,-8.391046e-02,-1.650696e-02,-2.131232e-02
25%,1.324961e-01,-3.335539e-01,-1.181562e+00,-1.097189e-01,-5.627739e-01,-9.328037e-02,-2.521937e-02,-1.088681e-01,-1.676285e-01,-4.007581e-01,...,-5.138291e-02,-9.529429e-03,-1.650696e-02,-1.062768e-01,-5.138291e-02,-1.183006e-01,-1.545757e-01,-8.391046e-02,-1.650696e-02,-2.131232e-02
50%,1.324961e-01,5.006528e-01,1.185655e-01,-1.097189e-01,-5.627739e-01,-9.328037e-02,-2.521937e-02,-1.088681e-01,-1.676285e-01,-4.007581e-01,...,-5.138291e-02,-9.529429e-03,-1.650696e-02,-1.062768e-01,-5.138291e-02,-1.183006e-01,-1.545757e-01,-8.391046e-02,-1.650696e-02,-2.131232e-02
75%,1.324961e-01,5.006528e-01,8.408588e-01,-1.097189e-01,-5.627739e-01,-9.328037e-02,-2.521937e-02,-1.088681e-01,-1.676285e-01,-4.007581e-01,...,-5.138291e-02,-9.529429e-03,-1.650696e-02,-1.062768e-01,-5.138291e-02,-1.183006e-01,-1.545757e-01,-8.391046e-02,-1.650696e-02,-2.131232e-02
max,1.324961e-01,1.334860e+00,1.852069e+00,9.114203e+00,1.776912e+00,1.072037e+01,3.965206e+01,9.185428e+00,5.965571e+00,2.495271e+00,...,1.946172e+01,1.049381e+02,6.058052e+01,9.409388e+00,1.946172e+01,8.453044e+00,6.469323e+00,1.191747e+01,6.058052e+01,4.692121e+01


In [ ]:
#TRAIN